In [ ]:
from os import sys
sys.path.append("../../meshAfterParty/")
#sys.path.append("../../meshAfterParty/meshAfterParty")

In [ ]:
from importlib import reload
import os
from pathlib import Path
os.getcwd()

import neuron_utils as nru
nru = reload(nru)
import neuron
neuron=reload(neuron)
import neuron_visualizations as nviz
import time

In [ ]:
import system_utils as su

# Loading the neuron object

In [ ]:
import system_utils as su
uncompressed_neuron = su.load_object("../test_neurons/test_objects/12345_2_soma_practice_decompress.pkl")
uncompressed_neuron = neuron.Neuron(uncompressed_neuron)

In [ ]:
ex_branch = uncompressed_neuron.concept_network.nodes["L1"]["data"].concept_network.nodes[0]["data"]
ex_branch.spines

# computeing the no_spine_average_mesh_center values

In [ ]:
with su.suppress_stdout_stderr():
    uncompressed_neuron.calculate_width_without_spines(
                                          skeleton_segment_size = 1000,
                                           width_segment_size=None,
                                          width_name = "no_spine_average_mesh_center")

In [ ]:
ex_branch.width_array

# Using the queries to find the mesh parts

In [ ]:
import neuron_visualizations as nviz
import neuron_searching as ns

In [ ]:
current_functions_list = [
    "spines_per_skeletal_length",
    "no_spine_average_mesh_center",
    "n_spines",
    "n_faces_branch",
    
]


with su.suppress_stdout_stderr():
    limb_branch_dict = ns.query_neuron(uncompressed_neuron,
                                       query="n_spines < 4 and no_spine_average_mesh_center < 400",
                   functions_list=current_functions_list)
    #print(F"limb_branch_dict BEFORE = {limb_branch_dict}")

    #How to give a limb_branch_dict as an argument
    current_functions_list = ["axon_segment"]
    limb_branch_dict_upstream_filter = ns.query_neuron(uncompressed_neuron,
                                       query="axon_segment==True",
                                       function_kwargs=dict(limb_branch_dict =limb_branch_dict,
                                                            downstream_face_threshold=3000,
                                                            width_match_threshold=50,
                                                           print_flag=False),
                                       functions_list=current_functions_list)

In [ ]:
nviz = reload(nviz)
with su.suppress_stdout_stderr():
    nviz.visualize_neuron(uncompressed_neuron,
                          visualize_type=["mesh"],
                         limb_branch_dict=limb_branch_dict_upstream_filter,
                          mesh_color="red",
                          mesh_color_alpha=1,
                         mesh_whole_neuron=True)

# Conclusions

In [ ]:
"""
There are still some false positives but this can be fixed when we do the merge analysis


"""

# How to find the axon from this point

In [ ]:
"""
Ideas: 
1) Can look at how much axon in a row without being interrupted (or only interrupted by a certain amount)
2) Look at branches that are within x distance of the soma
3) Look at the width profile of the branch
4) not a lot of spining
5) where on the soma it is
"""

# create a function that will measure the distance from the starting point (aka the soma)

In [ ]:
import neuron_searching as ns
ns = reload(ns)

current_functions_list = [
    "skeletal_distance_from_soma",
    "no_spine_average_mesh_center",
    "n_spines",
    "n_faces_branch",
    
]

function_kwargs=dict(somas=[0,1],print_flag=False)
soma_skeletal_distance_threshold = 30000
query="skeletal_distance_from_soma < @soma_skeletal_distance_threshold"
query_variables_dict = dict(limb_list=['L1'],
                           soma_skeletal_distance_threshold=soma_skeletal_distance_threshold)

# limb_branch_dict_df = ns.query_neuron(uncompressed_neuron,
#                                    query=query,
#                                       function_kwargs=function_kwargs,
#                                       query_variables_dict=query_variables_dict,
#                functions_list=current_functions_list,
#                                   return_dataframe=True)

limb_branch_dict = ns.query_neuron(uncompressed_neuron,
                                   query=query,
               functions_list=current_functions_list,
                                   query_variables_dict=query_variables_dict,
                                   function_kwargs=function_kwargs,
                                  return_dataframe=False)


In [ ]:
limb_branch_dict

In [ ]:
nviz = reload(nviz)

returned_colors = nviz.visualize_neuron(uncompressed_neuron,
                      visualize_type=["mesh"],
                     limb_branch_dict=limb_branch_dict,
                      mesh_color="red",
                      mesh_color_alpha=1,
                     #mesh_whole_neuron=True,
                     return_color_dict=True,
                    print_time=True)

In [ ]:
mu = reload(mu)
mu.plot_color_dict(returned_colors,print_flag=False)

# The algorithm for finding the axon branch

# --- practicing getting skeletons ----

In [ ]:
neuron = reload(neuron)
uncompressed_neuron = neuron.Neuron(uncompressed_neuron)

In [ ]:
ex_limb_obj = uncompressed_neuron.concept_network.nodes["L3"]["data"]

In [ ]:
ex_limb_obj.find_branch_by_skeleton_coordinate(np.array([729486.6, 977760.4, 921825.8]))

In [ ]:
ex_limb_obj.concept_network_data_by_soma
ex_limb_obj.get_skeleton_soma_starting_node(1)

In [ ]:
interested_soma = 0
ex_limb_obj = uncompressed_neuron.concept_network.nodes["L4"]["data"]

In [ ]:
ex_limb_obj.touching_somas()
ex_limb_starting_coordinate = ex_limb_obj.get_soma_starting_coordinate(1)
ex_limb_starting_coordinate

In [ ]:
import skeleton_utils as sk
import networkx as nx
import networkx_utils as xu
ex_limb_skeleton_graph = sk.convert_skeleton_to_graph(ex_limb_obj.skeleton)
xu.get_nodes_with_attributes_dict(ex_limb_skeleton_graph,
                                  attribute_dict=dict(coordinates=ex_limb_starting_coordinate))

In [ ]:


get_skeleton_soma_starting_node(uncompressed_neuron.concept_network.nodes["L5"]["data"],
                               0,
                               print_flag = True)

In [ ]:
ex_limb_skeleton_graph.nodes[256]

In [ ]:
ex_limb_obj.skeleton[1000][1]

In [ ]:
import numpy_utils as nu
import numpy as np


limb= ex_limb_obj
target_coordinate = np.array([695784., 986044., 906273.])



find_branch_by_skeleton_coordinate(limb= ex_limb_obj,
target_coordinate = np.array([729486.6, 977760.4, 921825.8]))

In [ ]:
"""
New Tool for help with identification

Purpose: 

Given: 
a) If given limb or branch (any object)
b) want starting endpoint of that limb/branch

Gete the skeleton of x distance from the starting point
Get the vector from the starting point to the endpoints
Get the angle between that and a hardcoded vector of your choice


Get the paths from the endpoints to the starting point
For all those paths get the vector from 



Better algorithm: 
1) Take all skeletons of branches that border a soma
and stitch them together at the soma node


Would want to trace it back to the specific branch it starts on

For each limb: 
1) 

2) 



"""

# Developing colors

In [ ]:
import webcolors
import numpy as np

def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

def convert_rgb_to_name(rgb_value):
    rgb_value = np.array(rgb_value)
    if not nu.is_array_like(rgb_value[0]):
        rgb_value = rgb_value.reshape(1,-1)
    
    print(f"rgb_value.shape = {rgb_value.shape}")

    output_colors = []
    for k in rgb_value:
        if len(k) > 3:
            k = k[:3]
        adjusted_color_value = np.array(k)*255
        output_colors.append(get_colour_name(adjusted_color_value)[-1])
    
    if len(output_colors) == 1:
        return output_colors[0]
    elif len(output_colors) > 1:
        return output_colors
    else:
        raise Exception("len(output_colors) == 0")